In [1]:
# Import the required modules and functions
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime
import zipfile
import numpy as np

In [2]:
with zipfile.ZipFile(r'C:\Users\user\Desktop\datasource.zip', 'r') as zip_ref:
    zip_ref.extractall('dealership_data')

In [3]:
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

In [4]:
# CSV Extract Function
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

In [5]:
# JSON Extract Function
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

In [6]:
# XML Extract Function
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["car_model", "year_of_manufacture", "price", "fuel"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for car in root:
        car_model = car.find("car_model").text
        year_of_manufacture = int(car.find("year_of_manufacture").text)
        price = float(car.find("price").text)
        fuel = car.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
    return dataframe

In [7]:
# Extract Function
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

In [8]:
def transform(data):
    np_array = np.array(data["price"]).astype(np.object)
    np_list = np.around(np_array.astype(np.double), 1)
    data["price"] = np_list
    return data

In [9]:
# Loading
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  

In [10]:
# Logging
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y' #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n') 

In [11]:
# Log that you have started the ETL process
log("ETL Job Started")

In [12]:
# Log that you have started the Extract step
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

C:\Users\user\AppData\Local\Temp\ipykernel_11460\1029713304.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_11460\1029713304.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_11460\1029713304.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_11460\1029713304.py:11: FutureWarning: The frame.append method is deprecated and will be removed fr

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.0,Petrol
1,sx4,2013,7089.552239,Diesel
2,ciaz,2017,10820.895522,Petrol
3,wagon r,2011,4253.731343,Petrol
4,swift,2014,6865.671642,Diesel
...,...,...,...,...
85,camry,2006,3731.343284,Petrol
86,land cruiser,2010,52238.80597,Diesel
87,corolla altis,2012,8805.970149,Petrol
88,etios liva,2013,5149.253731,Petrol


In [13]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

C:\Users\user\AppData\Local\Temp\ipykernel_11460\3377881791.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np_array = np.array(data["price"]).astype(np.object)


,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.0,Petrol
1,sx4,2013,7089.6,Diesel
2,ciaz,2017,10820.9,Petrol
3,wagon r,2011,4253.7,Petrol
4,swift,2014,6865.7,Diesel
...,...,...,...,...
85,camry,2006,3731.3,Petrol
86,land cruiser,2010,52238.8,Diesel
87,corolla altis,2012,8806.0,Petrol
88,etios liva,2013,5149.3,Petrol


In [14]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [15]:
log("ETL Job Ended")